In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd 
import numpy as np
from itertools import product, permutations

import numba
from numba import njit,vectorize, jit
from tqdm import tqdm

import Estimators

In [3]:
d = 2
c = 0

In [4]:
#@njit
sig2 = 1
@njit
def GaussianKernel(x,y):
    return np.exp(-np.linalg.norm(x-y)**2/(2*sig2))

@njit
def PolynomialKernel(x,y): 
    return (x.dot(y)+c)**d
@njit
def LinearKernel(x,y): 
    return x.dot(y)

@njit
def Laplace_kernel(x, y, gamma=1):
    return 0.5 * np.exp(-gamma * np.linalg.norm(x-y))

In [57]:
def importation_mat_100(): 
    X_train_100 = []
    X_test_100 = []
    Y_train = []
    for i in range(3): 
        xtrain = pd.read_csv('data/Xtr'+str(i)+'_mat100.csv',delimiter= ',', header= None)
        xtrain = np.squeeze(xtrain.to_numpy())
        X_train_100.append(xtrain)
    
        xtest = pd.read_csv('data/Xte'+str(i)+'_mat100.csv',delimiter= ',', header= None)
        xtest = np.squeeze(xtest.to_numpy())
        X_test_100.append(xtest)
    
        Y_train.append(pd.read_csv('data/Ytr'+str(i)+'.csv',delimiter= ',')['Bound'].to_numpy())
    return X_train_100,X_test_100,Y_train

def importation(): 
    X_train = []
    X_test = []
    Y_train = []
    for i in range(3): 
        xtrain = pd.read_csv('data/Xtr'+str(i)+'.csv',delimiter= ',', header= None)
        xtrain = xtrain.iloc[1:,1].to_numpy()
        X_train.append(xtrain)
    
        xtest = pd.read_csv('data/Xte'+str(i)+'.csv',delimiter= ',', header= None)
        xtest = xtest.iloc[1:,1].to_numpy()
        X_test.append(xtest)
    
        Y_train.append(pd.read_csv('data/Ytr'+str(i)+'.csv',delimiter= ',')['Bound'].to_numpy())
    return X_train,X_test,Y_train

X_train,X_test,Y_train = importation()
X_train_100, X_test_100,Y_train = importation_mat_100()

In [36]:
X_traintest = []
for i in range(3): 
    X_traintest.append(np.concatenate((X_train[i],X_test[i])))


In [45]:
def to_Kernel_train(X, Kernel): 
    length = X.shape[0]
    mat_K = np.zeros((length,length))
    for i in range(length):
        x_i = np.squeeze(X[i])
        for j in range(i,length): 
            x_j = np.squeeze(X[j])
            value = Kernel(x_i,x_j)
            mat_K[i,j] = value
            mat_K[j,i] = value 
    return mat_K

#@njit 
def to_Kernel_test(Xtrain,Xtest,Kernel):
    length_train = Xtrain.shape[0]
    length_test = Xtest.shape[0]
    bimat_K = np.zeros((length_train,length_test))
    for i in range(length_train):
        x_i = np.squeeze(Xtrain[i])
        for j in range(length_test): 
            x_j = np.squeeze(Xtest[j])
            value = Kernel(x_i,x_j)
            bimat_K[i,j] = value
    return bimat_K


def standardize(K): 
    U = np.full(K.shape,1/K.shape[0])
    I = np.eye(K.shape[0])
    return (I-U)@K@(I-U)

In [9]:
# Compute phi for Spectrum kernel
def phi_spectrum(x,k,U):
    """U: list of the sequences of size k to look for in x"""
    phi_spec = np.zeros(len(U))
    for j, u in enumerate(U):
        for i in range(len(x)-k+1):
            if x[i:i+k] == u:
                phi_spec[j] +=1
    return phi_spec

#Compute phi for Substring kernel
def l(i):
    return i[-1] - i[0] + 1

def I(k,n):
    I = set()
    for ele in permutations(range(0,n),k):
        I.add(tuple(sorted(list(ele))))
    return list(I)

def phi_substring(x,k,U,lamb=0.5): #fonctionne (testé avec l'exemple page 392)
    """U: list of the sequences of size k to look for in x"""
    phi = np.zeros(len(U))
    I_kn = I(k,len(x))
    for j, u in enumerate(U):
        for i in I_kn:
            x_i = "".join([x[idx]  for idx in i])
            if x_i==u:
                phi[j] += lamb**l(i)
    return phi


def make_dict_phi(X, phi,k):
    U = [''.join(letter) for letter in product('ACGT', repeat=k)]
    phi_dict = {seq:phi(seq,k,U) for seq in tqdm(X)}
    return phi_dict
    

# Compute K(x_i,x) 
def K_substring_2(x, y):
    """
    Compute K(x_i,x) with K a string kernel
    
    Parameters:
    Ne prend pas en argument X_train mais seulement phi_X_train(qui a été calculé précédement)
    et i .
    
    phi_X_train: list of phi(X_train) which is also needed to construct the matrix K 
    phi: the function phi
    k: hyperparameter of the spectrum kernel (size of sequences in U)
    """
    value = np.sum([dict_sub_traintest_2[str(x)] * dict_sub_traintest_2[str(y)]])  
    return value


In [37]:
##CONSTRUCTION DES DICTIONNAIRES. On fait ça pour eviter de les recalculer à chaque fois
dict_spectrum_traintest = [0]*7

#dict_spectrum_traintest[3] = make_dict_phi(X_traintest[0],phi_spectrum,3)
#dict_spectrum_traintest[4] = make_dict_phi(X_traintest[0],phi_spectrum,4)

#dict_spectrum_traintest[5] = make_dict_phi(np.concatenate(X_traintest),phi_spectrum,5)
#dict_spectrum_traintest[6] = make_dict_phi(np.concatenate(X_traintest),phi_spectrum,6)
#dict_sub_traintest_2 = make_dict_phi(X_traintest[0], phi_substring,2)

#dict_sub_traintest_3 = make_dict_phi(X_traintest[0], phi_substring,3)

100%|██████████| 9000/9000 [07:37<00:00, 19.67it/s]


In [92]:
def make_K_spectrum(k): 
    
    def K_spectrum(x,y): 
        value = np.sum([dict_spectrum_traintest[k][str(x)] * dict_spectrum_traintest[k][str(y)]])
        return value 
    return K_spectrum
K_spectrum_5 = make_K_spectrum(5)
K_spectrum_6 = make_K_spectrum(6)

In [88]:
def make_function(L): 
    def f(x,y): 
        return L
    return f 

test = make_function(3)
print(test(0,0))

3


In [85]:
dict_spectrum_traintest[k][X_train[0][1]]

array([0., 0., 0., ..., 0., 2., 3.])

In [50]:
def get_first_Kernels(Kernel): 
    '''
    Permet d'avoir les Kernels de train et de test, au moins pour tester le modèle, voir ce qu'il donne sur le premier dataset. 
    '''
    K_train = to_Kernel_train(X_train[0], Kernel)
    K_test = to_Kernel_test(X_train[0], X_test[0], Kernel)
    return K_train,K_test 

def get_list_Kernels(Kernel) : 
    '''
    Permet d'avoir la liste des Kernel_train et Kernel_test pour chaque deteset. renvoie deux listes : la premiere contient 
    les 3 premiers Kernel_train des 3 datasets. Le deuxieme renvoie les Kernel_test des 3 datasets. 
    
    Ne marche qu'avec les kernels qui prenent des strings en entree. 
    '''
    list_K_train = list()
    list_K_test = list()
    for i in tqdm(range(3)): 
        list_K_train.append(to_Kernel_train(X_train[i], Kernel))
        list_K_test.append(to_Kernel_test(X_train[i],X_test[i], Kernel))
    return list_K_train,list_K_test

def get_list_Kernels(Kernel) : 
    '''
    Permet d'avoir la liste des Kernel_train et Kernel_test pour chaque deteset. renvoie deux listes : la premiere contient 
    les 3 premiers Kernel_train des 3 datasets. Le deuxieme renvoie les Kernel_test des 3 datasets. 
    
    Ne marche qu'avec les kernels qui prenent des strings en entree. 
    '''
    list_K_train = list()
    list_K_test = list()
    for i in tqdm(range(3)): 
        list_K_train.append(to_Kernel_train(X_train[i], Kernel))
        list_K_test.append(to_Kernel_test(X_train[i],X_test[i], Kernel))
    return list_K_train,list_K_test

In [93]:
list_K_train_spectrum_6, list_K_test_spectrum_6 = get_list_Kernels(K_spectrum_6)  

100%|██████████| 3/3 [05:33<00:00, 111.33s/it]


In [95]:
list_K_train_spectrum_5, list_K_test_spectrum_5 = get_list_Kernels(K_spectrum_5)  

100%|██████████| 3/3 [05:21<00:00, 107.27s/it]


In [106]:
def grid_search_cv(model, Kernel_train, Y_train, parameters): 
    scores = list()
    for parameter in parameters : 
        print('We test : ', parameter)
        model.set_parameter(parameter)
        scores.append(model.cross_val(Kernel_train,Y_train, 5))
    arg_max = np.argmax(np.array(scores))
    print('best_param = ',parameters[arg_max])
    return parameters[arg_max]
KRR_spectrum_6 = Estimators.KRR(Kernel = K_spectrum_6)
parameters = np.linspace(0.001,1,12)
parameters
grid_search_cv(KRR_spectrum_6, list_K_train_spectrum_6[2], Y_train[2], parameters)

0it [00:00, ?it/s]

We test :  0.001


5it [00:03,  1.50it/s]
0it [00:00, ?it/s]

Score :  0.6775
We test :  0.09181818181818181


5it [00:00, 12.49it/s]
0it [00:00, ?it/s]

...

5it [00:00,  8.80it/s]
0it [00:00, ?it/s]

Score :  0.7
We test :  1.0


5it [00:00, 12.96it/s]

Score :  0.7105
best_param =  0.18263636363636362


0.18263636363636362

In [69]:
Kernel_train_Gaussian = to_Kernel_train(X_mat100_0,GaussianKernel)
Kernel_test_Gaussian = to_Kernel_test(X_mat100_0,X_mat100_0,GaussianKernel)

In [71]:
for i in range(10): 
    KRR_Gaussian = Estimators.KRR(Kernel = GaussianKernel, lam = 10**(-i))
    KRR_Gaussian.cross_val(list_K_train_spectrum_6[0], Y_train[0], 5)

5it [00:02,  2.26it/s]
0it [00:00, ?it/s]

Score :  0.519


5it [00:00,  8.98it/s]
0it [00:00, ?it/s]

Score :  0.519


5it [00:00, 13.85it/s]
0it [00:00, ?it/s]

...

Score :  0.519


5it [00:00, 14.08it/s]
0it [00:00, ?it/s]

Score :  0.519


5it [00:00, 12.80it/s]

Score :  0.519


In [72]:
list_K_test_spectrum_6[0]

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [19]:
Kernel_train_Linear = to_Kernel_train(X_mat100_0,LinearKernel)
Kernel_test_Linear = to_Kernel_test(X_mat100_0,X_mat100_0,LinearKernel)

<ipython-input-10-025997df8f8d>:12: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 1d, A))
  return x.dot(y)


In [39]:
def try_Kernel(Kernel,X,Kernel_train = 0,pre_computed = False): 
    '''
    Fonction qui permet de voir si on overfit sur le train set, ce qui est normal (normalement). Le gaussian, polynomial et Laplace Kernel overfittent bien. 
    les restes non ( Spectrum, String et Linear)
    mettre les kernel de train et de test en parametre si on veut eviter de les recalculer
    '''
    if pre_computed == False : 
        Kernel_train = to_Kernel_train(X,Kernel)
    regressor = Estimators.KRR(Kernel = Kernel)
    regressor.fit(Kernel_train,Y_train[0])
    pred = regressor.predict(Kernel_train)
    print('predictions-true : ', (pred-Y_train[0])[0:100])
    print('predictions : ', pred[0:100]*1)
    print('true : ', Y_train[0][0:100])
    print('SCORE : ', 1-np.sum(np.abs(pred-Y_train[0]))/pred.shape[0])

In [112]:
def download_results(model,list_Kernel_train,list_Kernel_test, parameters, name_dossier):
    Y_predicted = []
    for i in range(3): 
        model.set_parameter(parameters[i])
        model.fit(list_Kernel_train[i], Y_train[i])
        Y_predicted.append(model.predict(list_Kernel_test[i])*1)
        print('Model {} Predicted'.format(i))

    d = { 'Id' : np.arange(3000), 'Bound' : np.concatenate(Y_predicted)}
    out = pd.DataFrame(data=d)
    out.to_csv('predictions_KM'+name_dossier+'.csv', index=False)

In [113]:
best_parameters = [0.3636,0.6363,0.1826]
download_results(KRR_spectrum_6,list_K_train_spectrum_6,list_K_test_spectrum_6,best_parameters,'K_spectrum_6')

Model 0 Predicted
Model 1 Predicted
Model 2 Predicted
